In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [6]:
#Parameters
samplenum = 3
batchsize = samplenum
epochs = 50
input_size = 3
output_size = 180
learning_rate = 0.01
time_length = 60; #seconds

# Generate simulation
sim = d.PySimSeq('test2.sim', 60)
yseq = sim.compute(sim.p)
f = sim.f(sim.y, sim.ydot, sim.yddot, sim.p)
df = sim.df_dp(sim.y, sim.ydot, sim.yddot, sim.p)

#Sample targets only variables in z direction
x = np.zeros((samplenum,3))
x[:,2] = np.random.rand(samplenum)
x[:,1] = 2
p = np.zeros((samplenum, 3*time_length))
y = np.zeros((samplenum, 3*time_length))

for i in range(samplenum):
    p[i, :] = sim.sample_ptraj(x[i, :], sim.p)
    y[i, :] = sim.compute(p[i, :].transpose())
    
x= torch.tensor(x)
y= torch.tensor(y)

## Test 

In [21]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input, y_target):
        #print(input.shape)
        #print(y_target.shape)
        #ctx.mark_dirty( y_target)
        p = input.clone().numpy().transpose()
        y_target = y_target.clone().numpy()
        #print(p.shape)
        #print(y_target.shape)
        yTraj_pred = np.zeros((len(y_target[:, 1]),180))
        
        for i in range(len(y_target[:,1])):
            yTraj_pred[i, :] = sim.compute(p[:, i])

        yTraj_pred = torch.tensor(yTraj_pred)
        #print(f'yTraj_pred {yTraj_pred.shape}')
        
        ctx.save_for_backward(input, yTraj_pred)
        
        return yTraj_pred, input
    
    @staticmethod
    def backward(ctx, grad_output, input):
        input, yTraj_pred = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        yTraj_pred = yTraj_pred.clone().numpy()
        dy_dp = np.zeros((samplenum, 3*time_length))
        for i in range(len(yTraj_pred[:,1])):
            yTraj_pred[i, :] = sim.compute(p[:, i])
        dy_dp = sim.dy_dp(sim.y, sim.ydot, sim.yddot, p[:, i])
        #print(dy_dp)
        print(f'shape of grad_output: {grad_output.shape}')

        
        grad_input = torch.tensor(dy_dp).mm(grad_output.t()).t()
        return grad_input, None

Simulate = Simulate.apply

class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(input_size, output_size)
        self.P = nn.Linear(output_size, output_size)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        x, p = Simulate(x, input)
        return x, p

model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

x_train = x.float()
y_train = y.float()
y_target = x_train


In [22]:
start_time = time.time()

losses = []

for i in range(epochs):
    y_pred, p_pred = model(x_train)
    print(y_pred.dtype)
    print(x_train.dtype)
    #loss = torch.sqrt(criterion(y_pred, y)) # RMSE
    loss = criterion(y_pred, y) # RMSE
    losses.append(loss)

    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')

    #Clear the gradient buffer (w <-- w - lr*gradient)
    optimizer.zero_grad()
    #Back Prop
    loss.backward()
    optimizer.step()
    i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

torch.float64
torch.float32
epoch:   0/50  loss: 1.34852377
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   1/50  loss: 1.07913504
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   2/50  loss: 1.06361814
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   3/50  loss: 1.08703434
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   4/50  loss: 1.15299966
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   5/50  loss: 1.29066155
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   6/50  loss: 1.29729090
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   7/50  loss: 1.30667444
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   8/50  loss: 1.33294105
shape of grad_output: torch.Size([3, 180])
torch.float64
torch.float32
epoch:   9/50  loss: 1.22715179
shape of grad

In [28]:
#Test forward propagation

p_pred = p_pred.detach().numpy().transpose()
y_pred = y_pred.detach().numpy().transpose()
yTraj_test = sim.compute(p_pred)
print(np.sum(yTraj_test-y_pred))
#print(y_pred)

1.1368683772161603e-13
